In [58]:
%matplotlib inline
import matplotlib.pyplot as plt
import time
import numpy as np
from six import iteritems
import shelve
from scipy.ndimage.measurements import label
import h5py

import PIL
import os
from IPython.display import Image, display
from IPython.html.widgets import interact, fixed
from IPython.html import widgets
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

import h5py
from skimage import feature
import os
from os.path import expanduser
import time

import torch

from residuals import res_warp_img

def visualize_residuals(res, figsize=(10,10), x_coords=None, y_coords=None, vec_grid=50):
    res = res.squeeze()
    assert res.shape[0] == res.shape[1]
    plt.figure(figsize=figsize)
    n = res.shape[0]
    y, x = np.mgrid[0:n, 0:n]
    
    if x_coords is None:
        x_coords = [0, res.shape[0]]
    if y_coords is None:
        y_coords = [0, res.shape[1]]
    
    ex = (-1) * res[:, :, 0]
    ey = res[:, :, 1]
    r = np.arctan2(ex, ey)
    
    interval = (x_coords[1] - x_coords[0]) // vec_grid
    
    plt.quiver(  x[x_coords[0]:x_coords[1]:interval, y_coords[0]:y_coords[1]:interval],  
                 y[x_coords[0]:x_coords[1]:interval, y_coords[0]:y_coords[1]:interval],
                ex[x_coords[0]:x_coords[1]:interval, y_coords[0]:y_coords[1]:interval], 
                ey[x_coords[0]:x_coords[1]:interval, y_coords[0]:y_coords[1]:interval], 
                 r[x_coords[0]:x_coords[1]:interval, y_coords[0]:y_coords[1]:interval], alpha=0.6)
    plt.quiver(x[x_coords[0]:x_coords[1]:interval, y_coords[0]:y_coords[1]:interval],  
                 y[x_coords[0]:x_coords[1]:interval, y_coords[0]:y_coords[1]:interval],
                ex[x_coords[0]:x_coords[1]:interval, y_coords[0]:y_coords[1]:interval], 
                ey[x_coords[0]:x_coords[1]:interval, y_coords[0]:y_coords[1]:interval], edgecolor='k', facecolor='None', linewidth=.5)
    plt.gca().invert_yaxis()
    
def display_image(img, x_coords=None, y_coords=None, normalize=False, figsize=(10, 10), mask=False):
    if normalize and mask:
        raise Exception("Masks can't be normalized")
    
    plt.figure(figsize=figsize)
    
    if x_coords is None:
        x_coords = [0, img.shape[0]]
    if y_coords is None:
        y_coords = [0, img.shape[1]]
    
    if mask:
        plt.imshow(img[x_coords[0]:x_coords[1], y_coords[0]:y_coords[1]], cmap='gray', vmin=0, vmax=1.0)
    elif not normalize:
        plt.imshow(img[x_coords[0]:x_coords[1], y_coords[0]:y_coords[1]], cmap='gray', vmin=-0.5, vmax=0.5)
    else:
        plt.imshow(img[x_coords[0]:x_coords[1], y_coords[0]:y_coords[1]], cmap='gray')

class ZhenVisualizer():
    def __init__(self, img_path, vec_path):
        self.img_h5 = h5py.File(img_path, 'r')['main']
        self.vec_h5 = h5py.File(vec_path, 'r')['main']
        
    def display_multiimg(self, choice, i):
        if choice == 'Residual':
            visualize_residuals(self.vec_h5[i], figsize=(12, 12))
        elif choice == 'Source':
            display_image(self.img_h5[0, i], normalize=self.normalize, figsize=(12, 12))
        elif choice == 'Target':
            display_image(self.img_h5[0, i+1], normalize=self.normalize, figsize=(12, 12))   
        elif choice == 'Predicted Target':
            src_var = torch.FloatTensor(self.img_h5[0, i])
            vec_var = torch.FloatTensor(self.vec_h5[i])
            pred_tgt_var = res_warp_img(src_var, vec_var, is_pix_res=True)
            pred_tgt = pred_tgt_var.cpu().detach().numpy()
            display_image(pred_tgt, normalize=self.normalize, figsize=(12, 12))   
        else:
            raise Exception("bad choice bro")
        
    def visualize(self, normalize=True):
        id_selector = widgets.IntText(
            value=0,
            description='Sample ID:',
            disabled=False
        )
        
        self.normalize = normalize
        
        button_choice = widgets.ToggleButtons(
                options=['Source', 'Target', 'Predicted Target', 'Residual'],
                description='Image:',
                disabled=False,
                button_style='',
            )

        interact(self.display_multiimg, choice=button_choice, i=id_selector)

#  Dataset

In [59]:
z = ZhenVisualizer(img_path="./datasets/minnie_full_mip8_1300px_train.h5", vec_path='./datasets/vecs_for_zhen.h5')

In [60]:
z.visualize()

interactive(children=(ToggleButtons(description='Image:', options=('Source', 'Target', 'Predicted Target', 'Re…